### Models

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=722.00, M2=1957.00


In [3]:
only = gp.Model("only")
only.setParam('OutputFlag', 0)
only.setParam("MIPGap", 1e-7)

x_ind = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x_individual")
yp_ind = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus_individual")
ym_ind = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus_individual")

M = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")

only.update()

obj = gp.quicksum(P_DA[t] * x_ind[i, t] for i in range(I) for t in range(T)) + gp.quicksum(1 / S * (P_RT[t, s] * yp_ind[i, t, s] - P_PN[t] * ym_ind[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x_ind[i, t] == yp_ind[i, t, s] - ym_ind[i, t, s])
    only.addConstr(yp_ind[i, t, s] <= R[i, t, s])
    only.addConstr(yp_ind[i, t, s] <= M * z[i, t, s])
    only.addConstr(ym_ind[i, t, s] <= M * (1 - z[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"Objective value: {only.objVal}")
else:
    print("No optimal solution found.")
    
x_ind_vals = np.array([[x_ind[i, t].x for t in range(T)] for i in range(I)])
y_plus_ind_vals = np.array([[[yp_ind[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
y_minus_ind_vals = np.array([[[ym_ind[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-09
Objective value: 1543726.3813601604


In [4]:
agg = gp.Model("agg")
agg.setParam('OutputFlag', 0)
agg.setParam("MIPGap", 1e-7)

a_agg = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp_agg = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm_agg = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
prob = np.array([1 / S for s in range(S)])

agg.update()

obj = gp.quicksum(P_DA[t] * a_agg[t] for t in range(T)) + gp.quicksum(prob[s] * (P_RT[t, s] * bp_agg[t, s] - P_PN[t] * bm_agg[t, s]) for t in range(T) for s in range(S))
agg.setObjective(obj, GRB.MAXIMIZE)

for t, s in product(range(T), range(S)):
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a_agg[t] == bp_agg[t, s] - bm_agg[t, s])
    agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= bp_agg[t, s])
    agg.addConstr(bp_agg[t, s] <= M * z[t, s])
    agg.addConstr(bm_agg[t, s] <= M * (1 - z[t, s]))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print(f"Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")
    
a_agg_vals = np.array([a_agg[t].x for t in range(T)])
b_plus_ind_vals = np.array([[bp_agg[t, s].x for s in range(S)] for t in range(T)])
b_minus_ind_vals = np.array([[bm_agg[t, s].x for s in range(S)] for t in range(T)])

Objective value: 1567561.8840047559


In [ ]:
set = gp.Model("Settlement") 
set.setParam('OutputFlag', 0)
set.setParam("PoolSolutions", 15)
set.setParam("PoolSearchMode", 2)
set.setParam("PoolGap", 0.05)

a = set.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
zy = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
ze = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(1/S * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))

set.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
    set.addConstr(yp[i, t, s] <= R[i, t, s])
    set.addConstr(ep[i,t,s] == yp[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i))
    set.addConstr(em[i,t,s] == ym[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i))
    set.addConstr(gp.quicksum(d[i, j, t, s] for j in range(I) if j != i) <= yp[i, t, s])
    set.addConstr(gp.quicksum(d[j, i, t, s] for j in range(I) if j != i) <= ym[i, t, s])
    set.addConstr(d[i, i, t, s] == 0)
    set.addConstr(yp[i, t, s] <= M1 * zy[i, t, s])
    set.addConstr(ym[i, t, s] <= M1 * (1 - zy[i, t, s]))
    
for t, s in product(range(T), range(S)):
    set.addConstr(a[t] == gp.quicksum(x[i, t] for i in range(I)))
    set.addConstr(bp[t,s] == gp.quicksum(ep[i, t, s] for i in range(I)))
    set.addConstr(bm[t,s] == gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t] for i in range(I)) == gp.quicksum(ep[i, t, s] for i in range(I)) - gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= M2 * ze[t, s])
    set.addConstr(gp.quicksum(em[i, t, s] for i in range(I)) <= M2 * (1 - ze[t, s]))

set.optimize()

if set.status == GRB.OPTIMAL:
    print(f"Objective value: {set.objVal}")
else:
    print("No optimal solution found.")

Objective value: 1567561.8840047559


### Curve Derivation

Individual Participation Profit Curve 

Maximize Individual Profit

In [ ]:
i_target = 0
t_target = 10
x_max = 80
profits = []

for x_fixed in range(x_max + 1):
    m1 = gp.Model("x_vs_profit")
    m1.setParam("OutputFlag", 0)
    m1.setParam("MIPGap", 1e-7)

    x_ind = m1.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x_ind")
    yp_ind = m1.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus_ind")
    ym_ind = m1.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus_ind")
    z = m1.addVars(I, T, S, vtype=GRB.BINARY, name="z")

    m1.update()

    M = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
    obj = gp.quicksum(P_DA[t_target] * x_ind[i, t_target] for i in range(I)) + gp.quicksum(
        1 / S * (P_RT[t_target, s] * yp_ind[i, t_target, s] - P_PN[t_target] * ym_ind[i, t_target, s])
        for i in range(I) for s in range(S)
    )
    m1.setObjective(obj, GRB.MAXIMIZE)

    for i, s in product(range(I), range(S)):
        t = t_target
        m1.addConstr(R[i, t, s] - x_ind[i, t] == yp_ind[i, t, s] - ym_ind[i, t, s])
        m1.addConstr(yp_ind[i, t, s] <= R[i, t, s])
        m1.addConstr(yp_ind[i, t, s] <= M * z[i, t, s])
        m1.addConstr(ym_ind[i, t, s] <= M * (1 - z[i, t, s]))

    # x 고정
    m1.addConstr(x_ind[i_target, t_target] == x_fixed)

    m1.optimize()

    if m1.status == GRB.OPTIMAL:
        profits.append(m1.objVal)
    else:
        profits.append(None)

profits_array = np.array(profits, dtype=np.float64)
max_idx = np.nanargmax(profits_array)
max_val = profits_array[max_idx]

# 🎨 Plot
plt.figure(figsize=(12, 6))
plt.plot(range(x_max + 1), profits, ms=2, color='#140DCC')
plt.plot(max_idx, max_val, marker='o', color='red', markersize=7, label=f'Max: x={max_idx}, profit={max_val:.2f}')
plt.title(f"Expected Profit vs. Commitment (i={i_target}, t={t_target})")
plt.xlabel("Commitment x")
plt.ylabel("Expected Profit")
plt.grid(True)
plt.show()

Aggregated Participation Profit Curve

Maximize Aggregated Profit

In [39]:
i_target = 0
t_target = 10
x_max = 100

profits = []

for x_fixed in range(x_max + 1):
    m2 = gp.Model("Settlement_x_fixed")

    a = m2.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
    bp = m2.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
    bm = m2.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
    x = m2.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
    yp = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
    ym = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
    zy = m2.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")
    d = m2.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
    ep = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
    em = m2.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
    ze = m2.addVars(T, S, vtype=GRB.BINARY, name="z_e") 

    m2.update()

    obj = P_DA[t_target] * a[t_target] + gp.quicksum(
        1/S * (P_RT[t_target, s] * bp[t_target, s] - P_PN[t_target] * bm[t_target, s]) for s in range(S)
    )
    m2.setObjective(obj, GRB.MAXIMIZE)

    for i, s in product(range(I), range(S)):
        t = t_target
        m2.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
        m2.addConstr(yp[i, t, s] <= R[i, t, s])
        m2.addConstr(yp[i, t, s] <= M1 * zy[i, t, s])
        m2.addConstr(ym[i, t, s] <= M1 * (1 - zy[i, t, s]))
        m2.addConstr(ep[i,t,s] == yp[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i))
        m2.addConstr(em[i,t,s] == ym[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i))
        m2.addConstr(gp.quicksum(d[i, j, t, s] for j in range(I) if j != i) <= yp[i, t, s])
        m2.addConstr(gp.quicksum(d[j, i, t, s] for j in range(I) if j != i) <= ym[i, t, s])
        m2.addConstr(d[i, i, t, s] == 0)

    for s in product(range(S)):
        t = t_target
        m2.addConstr(a[t] == gp.quicksum(x[i, t] for i in range(I)))
        m2.addConstr(bp[t,s] == gp.quicksum(ep[i, t, s] for i in range(I)))
        m2.addConstr(bm[t,s] == gp.quicksum(em[i, t, s] for i in range(I)))
        m2.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t] for i in range(I)) ==
                    gp.quicksum(ep[i, t, s] for i in range(I)) - gp.quicksum(em[i, t, s] for i in range(I)))
        m2.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))
        m2.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= M2 * ze[t, s])
        m2.addConstr(gp.quicksum(em[i, t, s] for i in range(I)) <= M2 * (1 - ze[t, s]))

    # x 고정!
    m2.addConstr(x[i_target, t_target] == x_fixed)

    m2.optimize()

    if m2.status == GRB.OPTIMAL:
        profits.append(m2.objVal)
    else:
        profits.append(None)

# Plot!
plt.figure(figsize=(12, 6))
plt.plot(range(x_max + 1), profits, ms=2)
plt.title(f"Aggregated Profit vs. x[{i_target},{t_target}]")
plt.xlabel("Fixed x")
plt.ylabel("Aggregated Expected Profit")
plt.grid(True)
plt.show()


KeyError: (10, (0,))

Aggregated Participation Profit Curve

Maximize Individual Profit